### The Requirements:
Step 1: Choose an ETF with a minimum of 100 assets, identify those assets

Step 2: Retrieve historical data for your chosen ETF

Step 3: Calculate the price momentum factors for each asset in your ETF

Step 4: Using the price momentum factors, calculate the monthly z-factor score for each asset

Step 5: Identify long and short baskets (10 to 15 assets in each) using calculated z-factors

Step 6: Create a backtest to validate performance of your algorithm based on monthly restructuring over the previous 5 years.

Step 7: Chart:

1. Monthly portfolio return bar chart (pos/neg coloring) vs ETF

2. Monthly return for/ long picks vs short picks vs ETF

3. Cumulative portfolio return vs ETF

In [19]:
# Import Libraries
import pandas as pd
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt


In [20]:
# Step 1: Choose an ETF with a minimum of 100 assets, identify those assets
# etf = ["SPY"]

# Get the list of S&P 500 constituents
# SPY_tickers = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]['Symbol'].tolist()
tickers_list = pd.read_excel("https://www.ssga.com/us/en/intermediary/etfs/library-content/products/fund-data/etfs/us/holdings-daily-us-en-spy.xlsx", header=4).Ticker.dropna().to_list()

print(f'{len(tickers_list)} tickers')


504 tickers


In [21]:
# Step 2: Retrieve historical data for your chosen ETF

# GET ADJ CLOSE DATA FOR PREVIOUS 5 YEARS
data = yf.download(tickers_list, period = '5y')['Adj Close']
# DROP NA
sp500 = data.dropna(how= 'all', axis= 1)
# PRINT OUT TAIL
#sp500.tail()

[*********************100%%**********************]  504 of 504 completed


8 Failed downloads:
['TER', 'STE', 'L', 'SPGI', 'WRB']: TypeError("unsupported operand type(s) for |: 'dict' and 'dict'")
['BF.B']: Exception('%ticker%: No price data found, symbol may be delisted (period=5y)')
['-', 'BRK.B']: Exception('%ticker%: No data found, symbol may be delisted')


,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2023-11-20 00:00:00,113.980003,12.40,191.449997,138.309998,129.699997,101.199997,84.459999,330.899994,612.700012,183.820007,...,87.169998,60.060001,104.500000,29.940001,100.209999,127.516830,112.500000,220.539993,35.779999,176.059998
2023-11-21 00:00:00,123.919998,12.13,190.639999,138.729996,126.820000,101.910004,86.129997,330.260010,610.989990,181.250000,...,86.860001,60.020000,104.459999,30.990000,101.019997,127.636276,112.900002,222.410004,34.310001,178.729996
2023-11-22 00:00:00,123.989998,12.31,191.309998,138.470001,129.039993,102.699997,85.949997,333.130005,619.719971,182.669998,...,87.529999,60.430000,104.010002,31.360001,102.220001,128.332977,113.830002,225.860001,34.410000,179.350006
2023-11-24 00:00:00,126.620003,12.31,189.970001,138.669998,128.369995,102.870003,86.599998,334.040009,619.429993,183.050003,...,88.309998,60.520000,104.570000,31.320000,103.260002,127.904991,113.430000,228.919998,34.349998,180.210007
2023-11-27 00:00:00,125.010002,12.18,189.789993,139.089996,129.000000,102.709999,87.000000,332.429993,619.270020,183.770004,...,87.059998,60.450001,103.959999,31.480000,103.180000,127.690002,113.839996,231.500000,34.400002,178.789993


In [33]:
sp500[-252:]

,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2022-11-25,155.858887,14.50,147.286743,153.422043,97.669998,104.862244,58.730000,291.764496,334.299988,164.009949,...,74.346367,67.430794,109.472382,30.998371,112.042801,124.193222,118.303673,271.899994,49.282764,148.743179
2022-11-28,151.231583,13.83,143.418350,152.278244,95.300003,103.323036,58.020000,286.055206,328.970001,160.986725,...,77.590889,67.198372,106.184639,29.993979,108.202744,124.114700,116.775375,269.000000,47.835232,146.790985
2022-11-29,149.881119,14.14,140.385315,152.057190,95.379997,102.940674,58.980000,283.476196,326.779999,160.868912,...,79.664619,66.675423,106.890533,29.747801,108.469276,123.712273,118.095261,266.820007,48.597092,146.513519
2022-11-30,153.892776,14.43,147.207184,154.921463,102.139999,105.470078,59.910000,296.223663,344.929993,168.741104,...,83.008369,68.002159,107.664116,29.797037,110.907555,126.283882,119.186897,270.279999,49.349426,152.746674
2022-12-01,155.024780,13.98,147.485641,155.354019,101.269997,105.813217,59.759998,298.093933,344.109985,168.319000,...,83.385406,67.973106,107.141953,30.958984,112.338943,126.804100,122.412193,268.820007,49.511326,155.224060
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-11-20,113.980003,12.40,191.449997,138.309998,129.699997,101.199997,84.459999,330.899994,612.700012,183.820007,...,87.169998,60.060001,104.500000,29.940001,100.209999,127.516830,112.500000,220.539993,35.779999,176.059998
2023-11-21,123.919998,12.13,190.639999,138.729996,126.820000,101.910004,86.129997,330.260010,610.989990,181.250000,...,86.860001,60.020000,104.459999,30.990000,101.019997,127.636276,112.900002,222.410004,34.310001,178.729996
2023-11-22,123.989998,12.31,191.309998,138.470001,129.039993,102.699997,85.949997,333.130005,619.719971,182.669998,...,87.529999,60.430000,104.010002,31.360001,102.220001,128.332977,113.830002,225.860001,34.410000,179.350006


In [30]:
# window = 20 days
lag = 20

# 
returns = sp500.pct_change()
returns.tail()

,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2023-11-20 00:00:00,0.007335,0.008950,0.009278,0.000072,0.020055,0.016575,0.010287,0.009365,0.016660,0.004207,...,0.003453,-0.008256,-0.004383,0.008420,-0.006149,0.003603,0.007433,0.011559,-0.008040,0.007208
2023-11-21 00:00:00,0.087208,-0.021774,-0.004231,0.003037,-0.022205,0.007016,0.019773,-0.001934,-0.002791,-0.013981,...,-0.003556,-0.000666,-0.000383,0.035070,0.008083,0.000937,0.003556,0.008479,-0.041084,0.015165
2023-11-22 00:00:00,0.000565,0.014839,0.003514,-0.001874,0.017505,0.007752,-0.002090,0.008690,0.014288,0.007834,...,0.007714,0.006831,-0.004308,0.011939,0.011879,0.005458,0.008237,0.015512,0.002915,0.003469
2023-11-24 00:00:00,0.021211,0.000000,-0.007004,0.001444,-0.005192,0.001655,0.007563,0.002732,-0.000468,0.002080,...,0.008911,0.001489,0.005384,-0.001276,0.010174,-0.003335,-0.003514,0.013548,-0.001744,0.004795
2023-11-27 00:00:00,-0.012715,-0.010561,-0.000948,0.003029,0.004908,-0.001555,0.004619,-0.004820,-0.000258,0.003933,...,-0.014155,-0.001157,-0.005833,0.005109,-0.000775,-0.001681,0.003615,0.011270,0.001456,-0.007880


In [31]:
lagged_returns = sp500.shift(lag).pct_change()
lagged_returns.tail()

,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2023-11-20 00:00:00,0.001007,0.018953,0.000694,-0.010258,0.033479,-0.010333,-0.010875,-0.006936,-0.001017,-0.015914,...,0.025571,-0.000861,-0.014674,0.015385,-0.023527,-0.009921,0.005738,0.003618,0.003004,-0.003531
2023-11-21 00:00:00,-0.033839,-0.007086,0.002543,0.010917,0.013456,-0.010127,0.019057,0.003899,-0.001573,0.006469,...,0.017587,0.017743,-0.009685,0.009586,-0.002343,0.009683,-0.003043,0.003020,-0.006323,0.003724
2023-11-22 00:00:00,-0.021204,-0.015165,-0.013492,-0.007177,-0.028768,-0.013079,0.000839,-0.011517,-0.034139,-0.031165,...,-0.027128,0.006601,0.001845,-0.025727,-0.018783,0.003336,-0.011446,-0.033995,-0.007703,-0.020644
2023-11-24 00:00:00,0.008801,0.009964,-0.024606,-0.000413,-0.025570,0.004382,-0.012457,-0.002187,-0.013163,0.006696,...,-0.015741,0.005045,-0.009117,-0.035523,0.010597,-0.012966,-0.005017,0.029762,0.027675,-0.034215
2023-11-27 00:00:00,-0.014764,-0.020628,0.007969,-0.043182,-0.011951,-0.012024,-0.013220,-0.006848,-0.011978,-0.001803,...,-0.002970,-0.024427,-0.019052,-0.002608,-0.005863,0.005811,0.000679,0.011473,-0.032841,-0.012906


In [47]:
# PAST 52 WEEK
#last_52_week = sp500[-252:]
last_52_week = lagged_returns[-252:]

len(last_52_week)


252

In [40]:
returns / len(last_52_week) * 100

,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2018-11-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-11-29,0.000500,-0.005299,-0.003048,0.004328,NaN,0.002343,-0.000976,-0.002789,-0.000191,-0.005249,...,-0.016464,-0.000231,0.003085,-0.005814,0.000948,0.000562,0.000900,-0.000423,-0.000822,-0.000128
2018-11-30,0.004325,0.017972,-0.002144,0.019243,NaN,0.004279,0.003356,0.008500,0.002868,0.010639,...,0.004698,0.007085,0.002209,0.008475,0.009350,0.001555,0.007111,0.004193,0.003704,0.004705
2018-12-03,0.012725,-0.005039,0.013866,-0.002568,NaN,0.001179,-0.003189,0.005548,0.006912,0.006864,...,0.037687,0.003707,0.008585,0.000525,-0.003697,-0.000645,-0.001967,0.003465,-0.002202,-0.001268
2018-12-04,-0.009353,-0.029624,-0.017456,-0.013177,NaN,-0.014800,-0.006290,-0.006209,-0.014675,-0.013028,...,-0.031239,-0.000825,-0.008746,-0.010699,-0.023381,-0.005991,-0.015166,-0.022645,-0.020501,-0.012723
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-11-20,0.002911,0.003552,0.003682,0.000029,0.007958,0.006577,0.004082,0.003716,0.006611,0.001669,...,0.001370,-0.003276,-0.001739,0.003341,-0.002440,0.001430,0.002949,0.004587,-0.003190,0.002860
2023-11-21,0.034606,-0.008641,-0.001679,0.001205,-0.008812,0.002784,0.007846,-0.000767,-0.001108,-0.005548,...,-0.001411,-0.000264,-0.000152,0.013917,0.003208,0.000372,0.001411,0.003365,-0.016303,0.006018
2023-11-22,0.000224,0.005889,0.001395,-0.000744,0.006946,0.003076,-0.000829,0.003448,0.005670,0.003109,...,0.003061,0.002711,-0.001709,0.004738,0.004714,0.002166,0.003269,0.006156,0.001157,0.001377


In [53]:
from scipy.stats import linregress

def slopes_of_linear_regression(data):
    
    slopes_dict = {}

    # Create an array of numbers from 1 to trading_days for the x-axis
    x_values = np.arange(1, len(data) + 1)


    # Calculate the linear regression for each column
    for column in data.columns:
        y_values = data[column]
        slope, _, _, _, _ = linregress(x_values, y_values)
        # slopes_dict[column] = slope
        # Multiply the slope by 100 to express it as a percentage change
        slopes_dict[column] = slope * 100

    return slopes_dict


In [54]:
slopes = slopes_of_linear_regression(last_52_week)
print("Slopes for each column:", slopes)

Slopes for each column: {'A': -0.0019475730391944855, 'AAL': -0.0032622546462520987, 'AAPL': -0.0006854905493532623, 'ABBV': -0.000492344983955876, 'ABNB': -0.0006727761100973616, 'ABT': -0.001662320559917379, 'ACGL': -0.00205956086137558, 'ACN': -0.0004679459398815012, 'ADBE': -0.00039403870644101293, 'ADI': -0.0031927080916920123, 'ADM': -0.0001327084259899017, 'ADP': -0.0006919326654524321, 'ADSK': -0.00019323189941062194, 'AEE': -0.0015097563309432848, 'AEP': -0.0014010979246857971, 'AES': -0.0024336155774979137, 'AFL': -0.00048614363232064883, 'AIG': -0.00042376485855680267, 'AIZ': 0.0015626922821794004, 'AJG': -0.0005323687426052364, 'AKAM': 0.0008812494847866273, 'ALB': -0.002124131677344705, 'ALGN': -0.006409734789034335, 'ALK': -0.0036985394403329973, 'ALL': 0.0008008455274545216, 'ALLE': -0.0023262958150680624, 'AMAT': -0.002656778214716065, 'AMCR': -0.0011327369321127599, 'AMD': -0.0036357196845601043, 'AME': -0.0020481031518734274, 'AMGN': 0.0009865635013163823, 'AMP': -0.0

In [56]:
slopes

{'A': -0.0019475730391944855,
 'AAL': -0.0032622546462520987,
 'AAPL': -0.0006854905493532623,
 'ABBV': -0.000492344983955876,
 'ABNB': -0.0006727761100973616,
 'ABT': -0.001662320559917379,
 'ACGL': -0.00205956086137558,
 'ACN': -0.0004679459398815012,
 'ADBE': -0.00039403870644101293,
 'ADI': -0.0031927080916920123,
 'ADM': -0.0001327084259899017,
 'ADP': -0.0006919326654524321,
 'ADSK': -0.00019323189941062194,
 'AEE': -0.0015097563309432848,
 'AEP': -0.0014010979246857971,
 'AES': -0.0024336155774979137,
 'AFL': -0.00048614363232064883,
 'AIG': -0.00042376485855680267,
 'AIZ': 0.0015626922821794004,
 'AJG': -0.0005323687426052364,
 'AKAM': 0.0008812494847866273,
 'ALB': -0.002124131677344705,
 'ALGN': -0.006409734789034335,
 'ALK': -0.0036985394403329973,
 'ALL': 0.0008008455274545216,
 'ALLE': -0.0023262958150680624,
 'AMAT': -0.002656778214716065,
 'AMCR': -0.0011327369321127599,
 'AMD': -0.0036357196845601043,
 'AME': -0.0020481031518734274,
 'AMGN': 0.0009865635013163823,
 'AMP

In [26]:
# Slope is our (y2-y1)[change of price] 
# divided by (x1-x2)[change of 1 year]
open_price = sp500.iloc[0] # first data entry
close_price = sp500.iloc[-1] # most recent data entry
price_difference = close_price - open_price
# Since our change of time is 1 year, im just going to divide by count of rows
slope = price_difference / len(sp500) * 100
slope

A        4.462650
AAL     -2.079584
AAPL    11.644626
ABBV     5.463250
ABNB          NaN
          ...    
YUM      3.488368
ZBH      0.528890
ZBRA     4.247414
ZION    -0.519316
ZTS      7.087330
Length: 496, dtype: float64

In [43]:
price_difference

A        56.095512
AAL     -26.140366
AAPL    146.372952
ABBV     68.673058
ABNB           NaN
           ...    
YUM      43.848785
ZBH       6.648148
ZBRA     53.389999
ZION     -6.527798
ZTS      89.087738
Length: 496, dtype: float64

In [24]:
def calculate_slope(data):
    x = np.arange(len(data))
    y = data.values
    slope, _ = np.polyfit(x, y, 1)
    return slope

In [25]:

lag = 20

slope_52_week_trend = sp500.rolling(window=252).apply(calculate_slope, raw=False).shift(lag)
slope_52_week_trend

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/alex/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/6y/0l3pbhfs1bg6l9tz9cbfv4940000gp/T/ipykernel_36924/2370690624.py", line 3, in <module>
    slope_52_week_trend = sp500.rolling(window=52).apply(calculate_slope, raw=False).shift(lag)
  File "/Users/alex/opt/anaconda3/lib/python3.8/site-packages/pandas/core/window/rolling.py", line 1913, in apply
    return super().apply(
  File "/Users/alex/opt/anaconda3/lib/python3.8/site-packages/pandas/core/window/rolling.py", line 1390, in apply
    return self._apply(
  File "/Users/alex/opt/anaconda3/lib/python3.8/site-packages/pandas/core/window/rolling.py", line 615, in _apply
    return self._apply_blockwise(homogeneous_func, name, numeric_only)
  File "/Users/alex/opt/anaconda3/lib/python3.8/site-packages/pandas/core/window/rolling.py", line 490, in _apply_blockwise

1. **Slope of 52-week Trend Line (20-day lag):**
   - This indicator likely involves calculating the slope of the trend line based on the past 52 weeks of price data, with a 20-day lag. The slope provides information about the direction and strength of the trend.

In [ ]:
lag = 20

momentum_factors = returns.rolling(window=lag).mean()
momentum_factors

,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2018-11-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-11-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-11-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-11-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-11-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-11-17,0.001964,0.005400,0.004785,-0.002691,0.004793,0.001472,0.000724,0.005009,0.005594,0.004916,...,-0.000003,0.002166,-0.002287,-0.003148,0.004990,0.003166,0.003419,0.003578,0.010352,0.002600
2023-11-20,0.002281,0.004899,0.005214,-0.002174,0.004121,0.002817,0.001782,0.005824,0.006478,0.005922,...,-0.001109,0.001796,-0.001773,-0.003497,0.005858,0.003842,0.003503,0.003975,0.009800,0.003137
2023-11-21,0.008333,0.004165,0.004876,-0.002568,0.002338,0.003674,0.001818,0.005532,0.006417,0.004900,...,-0.002166,0.000875,-0.001308,-0.002222,0.006380,0.003405,0.003833,0.004248,0.008062,0.003709


In [ ]:
# Step 3: Calculate the price momentum factors for each asset in your ETF

def calculate_momentum_factors(data, lag=20):
    # Factor 1: Slope of 52-week trend line (20-day lag)
    data['Slope_52Week'] = data['Close'].pct_change(252 - lag).rolling(window=20).mean() * 100

    # Factor 2: Percent above 260-day low (20-day lag)
    data['Percent_Above_260Day_Low'] = (data['Close'] - data['Low'].rolling(window=260 - lag).min()) / (data['High'].rolling(window=260 - lag).max() - data['Low'].rolling(window=260 - lag).min()) * 100

    # Factor 3: 4/52 Week Price Oscillator (20-day lag)
    data['Price_Oscillator'] = (data['Close'].rolling(window=4).mean() / data['Close'].rolling(window=52 - lag).mean() - 1) * 100

    # Factor 4: 39-week return (20-day lag)
    data['39Week_Return'] = data['Close'].pct_change(39 - lag) * 100

    # Factor 5: 51-week Volume Price Trend (20-day lag)
    data['Volume_Price_Trend'] = (data['Close'].pct_change() * data['Volume']).rolling(window=51 - lag).sum()

    return data[['Slope_52Week', 'Percent_Above_260Day_Low', 'Price_Oscillator', '39Week_Return', 'Volume_Price_Trend']]

# Apply the function to ETF data and asset data
#etf_momentum_factors = calculate_momentum_factors(data)
all_asset = {}

for asset in tickers_list:
    assets_momentum_factors = data[asset].apply(calculate_momentum_factors)

# Display the calculated price momentum factors
# print("ETF Momentum Factors:")
# print(etf_momentum_factors.head())
print("\nAsset Momentum Factors:")
print(assets_momentum_factors.head())

# # Displaying the signals for each asset
# for asset, signals in assets_momentum_factors.items():
#     print(f"Signals for {asset}:\n{signals}\n")



TypeError: 'float' object is not subscriptable

In [ ]:
# import yfinance as yf
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt

# # Function to calculate the slope of a trend line
# def calculate_slope(data):
#     x = np.arange(len(data))
#     y = data.values
#     slope, _ = np.polyfit(x, y, 1)
#     return slope

# # Function to calculate the slope of the 52-week trend line with a 20-day lag
# def calculate_52_week_slope(ticker, start_date, end_date, lag):
#     # Download historical data
#     df = yf.download(ticker, start=start_date, end=end_date)
    
#     # Calculate the closing prices
#     closing_prices = df['Close']

#     # Calculate the slope with a 52-week window and a 20-day lag
#     slope_52_week_trend = closing_prices.rolling(window=52).apply(calculate_slope, raw=False).shift(lag)
    
#     return slope_52_week_trend

# # Example usage
# ticker = "AAPL"
# start_date = "2018-01-01"
# end_date = "2023-01-01"
# lag = 20

# # Calculate the 52-week slope with a 20-day lag
# slope_52_week_trend = calculate_52_week_slope(ticker, start_date, end_date, lag)

# # Plotting the results
# plt.figure(figsize=(10, 6))
# plt.plot(slope_52_week_trend, label='52-Week Slope (20-Day Lag)')
# plt.title(f'52-Week Slope of {ticker} Stock Price with a 20-Day Lag')
# plt.xlabel('Date')
# plt.ylabel('Slope')
# plt.legend()
# plt.show()
